In [54]:
import numpy as np
import pandas as pd
import os

print(os.listdir('/content/sample_data'))

def read_data_from_csv(file_path):
    data = pd.read_csv(file_path)
    supply = data['Supply'].values.tolist()
    demand = data['Demand'].values.tolist()
    costs = data.drop(columns=['Supply', 'Demand']).values
    return supply, demand, costs

file_path = '/content/sample_data/test.csv'

supply, demand, costs = read_data_from_csv(file_path)

print("Supply:", supply)
print("Demand:", demand)
print("Costs:", costs)

def balance_problem(supply, demand, costs):
    """Équilibrer le problème en ajoutant une variable auxiliaire si nécessaire."""
    supply_total, demand_total = sum(supply), sum(demand)
    if supply_total > demand_total:
        demand.append(supply_total - demand_total)
        costs = np.column_stack((costs, np.full(len(supply), np.inf)))
    elif demand_total > supply_total:
        supply.append(demand_total - supply_total)
        costs = np.vstack((costs, np.full(len(demand), np.inf)))
    return supply, demand, costs

def northwest_corner_method(supply, demand):
    """Méthode du coin nord-ouest pour initialiser une solution faisable de base."""
    rows, cols = len(supply), len(demand)
    allocation = np.zeros((rows, cols))
    i, j = 0, 0
    while i < rows and j < cols:
        allocated = min(supply[i], demand[j])
        allocation[i, j] = allocated
        supply[i] -= allocated
        demand[j] -= allocated

        if supply[i] == 0:
            i += 1
        elif demand[j] == 0:
            j += 1

    return allocation

def vogels_method(cost, supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    allocation = np.zeros_like(cost)

    while np.sum(supply_copy) > 0 and np.sum(demand_copy) > 0:
        row_penalties = []
        for i in range(len(supply)):
            if supply_copy[i] <= 0:
                row_penalties.append(-1)
                continue
            row_costs = [cost[i, j] for j in range(len(demand)) if demand_copy[j] > 0]
            if len(row_costs) < 2:
                row_penalties.append(0)
            else:
                row_costs.sort()
                row_penalties.append(row_costs[1] - row_costs[0])

        col_penalties = []
        for j in range(len(demand)):
            if demand_copy[j] <= 0:
                col_penalties.append(-1)
                continue
            col_costs = [cost[i, j] for i in range(len(supply)) if supply_copy[i] > 0]
            if len(col_costs) < 2:
                col_penalties.append(0)
            else:
                col_costs.sort()
                col_penalties.append(col_costs[1] - col_costs[0])

        max_row_penalty = max(row_penalties)
        max_col_penalty = max(col_penalties)

        if max_row_penalty >= max_col_penalty:
            i = row_penalties.index(max_row_penalty)
            min_cost = np.inf
            min_j = -1
            for j in range(len(demand)):
                if demand_copy[j] > 0 and cost[i, j] < min_cost:
                    min_cost = cost[i, j]
                    min_j = j
        else:
            j = col_penalties.index(max_col_penalty)
            min_cost = np.inf
            min_i = -1
            for i in range(len(supply)):
                if supply_copy[i] > 0 and cost[i, j] < min_cost:
                    min_cost = cost[i, j]
                    min_i = i
            i = min_i
            min_j = j

        allocated = min(supply_copy[i], demand_copy[min_j])
        allocation[i, min_j] = allocated
        supply_copy[i] -= allocated
        demand_copy[min_j] -= allocated

    return allocation

def calculate_total_cost(allocation, cost):
    return np.sum(allocation * cost)


if __name__ == "__main__":
    file_path = '/content/sample_data/test.csv'
    supply, demand, costs = read_data_from_csv(file_path)

    methods = {
        "Northwest Corner Rule": northwest_corner_method,
        "Vogel's Method": vogels_method
    }

    print("\ninitial feasible solutions :")


    for method_name, method_func in methods.items():
        if method_name == "Vogel's Method":
            allocation = method_func(costs, supply, demand)
        else:
            allocation = method_func(supply, demand)

        total_cost = calculate_total_cost(allocation, costs)
        print(f"\n{method_name}:")
        print(f"Allocation:\n{allocation}")
        print(f"Total Cost: {total_cost}")


['README.md', 'anscombe.json', 'test.csv', '.ipynb_checkpoints', 'mnist_train_small.csv', 'mnist_test.csv', 'california_housing_test.csv', 'california_housing_train.csv']
Supply: [20, 30, 50]
Demand: [30, 40, 30]
Costs: [[ 8  6 10]
 [ 9 12 13]
 [14  9 16]]

initial feasible solutions :

Northwest Corner Rule:
Allocation:
[[20.  0.  0.]
 [10. 20.  0.]
 [ 0. 20. 30.]]
Total Cost: 1150.0

Vogel's Method:
Allocation:
[[0 0 0]
 [0 0 0]
 [0 0 0]]
Total Cost: 0
